In [33]:
import os
import itertools
import numpy as np
import subprocess
import pandas as pd

In [2]:
wd = "d:/Projects/Tools/DayCent-CUTE/tools/test01/dream_worker001/"

In [3]:
os.chdir(wd)

In [4]:
fnam = "DayCentRUN.DAT"

In [5]:
with open(fnam) as f:
    for txtline in itertools.islice(f, 1, None):  # start=1, stop = None
        txtline = txtline.split()
#         if len(txtline) == 0:
#             parm.fnam1.close()
#             parm.iflg = 1

        print(txtline)

['wooster', 'wooster_cb_ct', 'wooster_cb_ct', 'wooster_base']
['wooster', 'wooster_ccc_nt', 'wooster_ccc_nt', 'wooster_base']
['wooster', 'wooster_cb_nt', 'wooster_cb_nt', 'wooster_base']


In [ ]:
with open('"DayCentRUN.DAT"', "r") as f:
    updated = []
    for line in f.readlines():
        parnam = line.split()[1].strip().replace("'", "").replace(",", "_")
        if (parnam in dc_parms.loc[:, "name"].tolist()):
            new_val = dc_parms.loc[dc_parms['name'] == parnam, "val"].tolist()[0]
            new_line = f"{new_val:<14.4f}{line.split()[1].strip()}\n"
            updated.append(new_line)
        else:
            updated.append(line)

In [16]:
data[0][0]

'wooster'

In [12]:
np.shape(data)

(4, 4)

In [17]:
type(data)

list

In [24]:
os.getcwd()

'd:\\Projects\\Tools\\DayCent-CUTE\\tools\\test01\\dream_worker001'

In [30]:
with open("DayCentRUN.DAT", "r") as f:
    data = [x.strip().split() for x in f]
for i in range(len(data)):
    if os.path.isfile(data[i][1]+".bin"):
        os.remove(data[i][1]+".bin")
    comline = 'DD17centEVI.exe -s {} -n {} -e {}'.format(data[i][1], data[i][1], data[i][3])
    run_model = subprocess.Popen(comline, cwd=".")
    run_model.wait()
    comline2 = 'DD17list100.exe {} {} {}'.format(data[i][1], data[i][1], 'outvars.txt')
    extract_model = subprocess.Popen(comline2, cwd=".")
    extract_model.wait()
print('  Simulation complete ...')

  Simulation complete ...


In [42]:
with open("DayCentRUN.DAT", "r") as f:
    data = [x.strip().split() for x in f]
    
# get simulation output dataset
data
    
    

[['wooster', 'wooster_ccc_ct', 'wooster_ccc_ct', 'wooster_base'],
 ['wooster', 'wooster_cb_ct', 'wooster_cb_ct', 'wooster_base'],
 ['wooster', 'wooster_ccc_nt', 'wooster_ccc_nt', 'wooster_base'],
 ['wooster', 'wooster_cb_nt', 'wooster_cb_nt', 'wooster_base']]

In [19]:
sim_df = pd.DataFrame()


# NOTE: temp inputs
outf = "wooster_ccc_nt.lis"
df = pd.read_csv( outf, sep=r'\s+', skiprows=1)
df_sel = df.loc[(df['time']>=1971.08) & (df['time']=<1994.00)]
df_sel.index = pd.date_range(start='1/1/1971', periods=len(df_sel), freq='M')
dfa = df_sel.resample('A').mean()
dfa.index = dfa.index.year


obd = read_observed_data()
comb = pd.concat([dfa, obd], axis=1)
comb.dropna(axis=0, inplace=True)
sim_list = comb.iloc[:,1].tolist() + comb.iloc[:,0].tolist()
return sim_list

[['wooster', 'wooster_ccc_ct', 'wooster_ccc_ct', 'wooster_base'],
 ['wooster', 'wooster_cb_ct', 'wooster_cb_ct', 'wooster_base'],
 ['wooster', 'wooster_ccc_nt', 'wooster_ccc_nt', 'wooster_base'],
 ['wooster', 'wooster_cb_nt', 'wooster_cb_nt', 'wooster_base']]

In [21]:
len(a)

3

In [49]:
for i in range(len(data)):
    print(data[i][1][8:])

ccc_ct
cb_ct
ccc_nt
cb_nt


In [97]:
# get all sims
sim_df = pd.DataFrame()
for i in range(len(data)):
    outf = data[i][1]+".lis"
    df = pd.read_csv( outf, sep=r'\s+', skiprows=1)
    df_sel = df.loc[(df['time']>=1971.08) & (df['time']<=1994.00)]
    df_sel.index = pd.date_range(start='1/1/1971', periods=len(df_sel), freq='M')
    dfa = df_sel.loc[:, ['somsc']].resample('A').mean()
    dfa.index = dfa.index.year
    dfa.rename(columns = {'somsc':'somsc_'+data[i][1][8:]}, inplace = True)
    sim_df = pd.concat([sim_df, dfa], axis=1)
    
# get all obds
obd_f = "soc_obd.csv"
obd_df = pd.read_csv(obd_f).set_index('Year')  
obd_df.replace(-999, np.nan, inplace=True)

# filter only for calibration
tot_df = pd.DataFrame()
for col in sim_df.columns:
    tt =  pd.concat([sim_df.loc[:, col], obd_df.loc[:, col]], axis=1).dropna(axis=0)
    tt.columns = ['sim', 'obd']
    tt['name'] = col
    tot_df = pd.concat([tot_df, tt], axis=0)



In [98]:
tot_df

,sim,obd,name
1971,3264.180917,3493.0,somsc_ccc_ct
1980,3505.868833,2805.0,somsc_ccc_ct
1992,3441.234833,3210.0,somsc_ccc_ct
1980,3212.758750,2370.0,somsc_cb_ct
1992,3208.799917,2234.0,somsc_cb_ct
1971,3576.392667,4230.0,somsc_ccc_nt
1980,4195.852083,3500.0,somsc_ccc_nt
1992,4545.282833,4250.0,somsc_ccc_nt
1980,3819.261583,3260.0,somsc_cb_nt
1992,4179.870000,3673.0,somsc_cb_nt


In [91]:
obd_f = "soc_obd.csv"
obd_df = pd.read_csv(obd_f).set_index('Year')  
obd_df.replace(-999, np.nan, inplace=True)
obd_df

,somsc_cb_nt,somsc_cb_ct,somsc_ccc_ct,somsc_ccc_nt
Year,,,,
1971,NaN,NaN,3493,4230
1980,3260.0,2370.0,2805,3500
1992,3673.0,2234.0,3210,4250


In [95]:
tot_df = pd.DataFrame()
for col in sim_df.columns:
    tt =  pd.concat([sim_df.loc[:, col], obd_df.loc[:, col]], axis=1).dropna(axis=0)
    tt.columns = ['sim', 'obd']
    tt['name'] = col
    tot_df = pd.concat([tot_df, tt], axis=0)


              sim     obd          name
1971  3264.180917  3493.0  somsc_ccc_ct
1980  3505.868833  2805.0  somsc_ccc_ct
1992  3441.234833  3210.0  somsc_ccc_ct
              sim     obd         name
1980  3212.758750  2370.0  somsc_cb_ct
1992  3208.799917  2234.0  somsc_cb_ct
              sim     obd          name
1971  3576.392667  4230.0  somsc_ccc_nt
1980  4195.852083  3500.0  somsc_ccc_nt
1992  4545.282833  4250.0  somsc_ccc_nt
              sim     obd         name
1980  3819.261583  3260.0  somsc_cb_nt
1992  4179.870000  3673.0  somsc_cb_nt


In [96]:
tot_df

,sim,obd,name
1971,3264.180917,3493.0,somsc_ccc_ct
1980,3505.868833,2805.0,somsc_ccc_ct
1992,3441.234833,3210.0,somsc_ccc_ct
1980,3212.758750,2370.0,somsc_cb_ct
1992,3208.799917,2234.0,somsc_cb_ct
1971,3576.392667,4230.0,somsc_ccc_nt
1980,4195.852083,3500.0,somsc_ccc_nt
1992,4545.282833,4250.0,somsc_ccc_nt
1980,3819.261583,3260.0,somsc_cb_nt
1992,4179.870000,3673.0,somsc_cb_nt


In [55]:
dfa.rename('hi')

1971    3576.392667
1972    3649.640583
1973    3724.723667
1974    3802.821000
1975    3874.368250
1976    3951.018750
1977    4011.787083
1978    4076.274583
1979    4142.703917
1980    4195.852083
1981    4230.657500
1982    4252.585333
1983    4259.157083
1984    4256.192083
1985    4270.419750
1986    4290.811917
1987    4327.455583
1988    4377.204167
1989    4416.286750
1990    4458.762750
1991    4499.521500
1992    4545.282833
1993    4577.368750
Name: hi, dtype: float64

In [116]:
df_fix = pd.read_csv('dc_fix.parms.csv')
df_site = pd.read_csv('dc_site.parms.csv')

In [117]:
dff = pd.concat([df_fix, df_site], axis=0)

In [118]:
dff

,name,select,default,min,max
0,ANEREF(1),1,1.500000,1.000000,2.0000
1,ANEREF(2),1,3.000000,2.800000,5.0000
2,ANEREF(3),1,1.000000,0.200000,1.1000
3,DAMR(1_1),1,0.200000,0.002000,0.3000
4,DAMRMN(1),1,15.000000,5.000000,30.0000
5,DEC1(1),1,3.900000,3.000000,5.0000
6,DEC1(2),1,4.900000,3.000000,7.0000
7,DEC2(1),1,14.800000,12.000000,18.0000
8,DEC2(2),1,18.500000,16.000000,21.0000
9,DEC3(1),1,6.000000,4.000000,8.0000


In [135]:
# for i in range(len(dff)):
#     dff.loc[i, 'val'] = 'cha'
dff['val'] = 50

with open('fix.100', "r") as f:
    updated = []
    for line in f.readlines():
        parnam = line.split()[1].strip().replace("'", "").replace(",", "_")
        if (parnam in dff.loc[:, "name"].tolist()):
            new_val = dff.loc[dff['name'] == parnam, "val"].tolist()[0]
            new_line = f"{new_val:<14.4f}{line.split()[1].strip()}\n"
            updated.append(new_line)
        else:
            updated.append(line)

with open('fix.100', "w") as wf:
    wf.writelines(updated)
    
    
with open('sitepar.in', "r") as f:
    updated = []
    for line in f.readlines():
        parnam = line.split()[2].strip()
        if (parnam in dff.loc[:, "name"].tolist()):
            new_val = dff.loc[dff['name'] == parnam, "val"].tolist()[0]
            new_line = f"{new_val:<11f}{line.split()[1].strip()} {line.split()[2].strip()}\n"
            updated.append(new_line)
        else:
            updated.append(line)

with open('sitepar.in', "w") as wf:
    wf.writelines(updated)

In [132]:
len(dff)

30

In [125]:
with open('sitepar.in', "r") as f:

SyntaxError: incomplete input (1639655778.py, line 1)